In [1]:
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


import PIL
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical

import warnings
warnings.filterwarnings("ignore")
import pandas as pd2
import numpy as np
from sklearn.datasets import load_linnerud
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.pipeline import FeatureUnion
from physlearn import Regressor
import pandas as pd

In [128]:
df = pd.read_excel("data_1.xlsx")

In [67]:
df.sample(3)

,DOI,Date,Journal,Title,Name,measurement_error,measurement_wavelength,measurement_method,normalised_name,raw_value,specifier
883,10.1016/S0166-1280(02)00027-1,2/1/2002,Journal of Molecular Structure: THEOCHEM,ATHEORETICALEXPERIMENTALSTUDYSOLUBILITYDISSOLU...,flavone,0.0,NaN,el_mylogic,O=C1C=C(Oc2ccccc12)c3ccccc3,1.851775,n
3795,10.1016/j.jlumin.2016.12.049,1/5/2017,Journal of Luminescence,EXPLORATIONSPECTROSCOPICPROPERTIESSOLVATEDTRIS...,Toluene,0.0,NaN,el_cde_tables,NaN,1.4969,Refractive index
2625,10.1016/j.optmat.2012.09.008,11/20/2012,Optical Materials,FLUORESCENCESPECTROSCOPICSTUDIESMN2IONSINSROAL...,MnO,0.0,NaN,el_mylogic,NaN,1.5269,Refractive index (nd)


In [37]:
# Для удобста ватаскивания значений через iloc
column_index_dic = {col_name:index for index,col_name in enumerate(df.columns)}

In [112]:
# В столбце "Name" есть пропущенные значения - 4 штуки 
print("Missed values ['Name']: ", df["Name"].isna().sum())
# В столбце "Name", заполним их исходя их столбца "normalized name"

import requests 

def convert_smiles_to_name(smi):
    try:
        url ="https://cactus.nci.nih.gov/chemical/structure/" + smi+"/iupac_name" 
        res = requests.get(url)
        return res.text.strip()
    except:
        return 'name not found'
 

for index in df[df["Name"].isna()].index:
    df.iloc[index,4] = convert_smiles_to_name(df.iloc[index,column_index_dic["normalised_name"]])


Missed values ['Name']:  4


In [124]:
# Теперь заполним пропуски в "normalised_name" через "name"

# В столбце "Name" есть пропущенные значения - 4 штуки 
print("Missed values ['normalised_name']: ", df["normalised_name"].isna().sum())
# В столбце "Name", заполним их исходя их столбца "normalized name"

import requests 

def convert_names_to_smiles(norm_name):
    try:
        url ="https://cactus.nci.nih.gov/chemical/structure/" + norm_name+"/smiles" 
        res = requests.get(url)
        return res.text.strip()
    except:
        return 'name not found'
 

for index in df[df["normalised_name"].isna()].index:
    df.iloc[index,column_index_dic["normalised_name"]] = convert_names_to_smiles(df.iloc[index,column_index_dic["Name"]])


Missed values ['normalised_name']:  2078


In [412]:
df = pd.read_excel("./data_partially_mod.xlsx")

In [414]:
# Выяснятется, что не все значения smiles были получены, 575 остается пропущенными
# повторно пробегаемся, по тем, что не получилось достать
sum(df["normalised_name"]=="<h1>Page not found (404)</h1>") 

# from rdkit import Chem

# # The string representation of the molecule
# mol_string = "B2O3–TeO2"

# # Convert the string representation to a molecule object
# mol = Chem.MolFromSmiles(mol_string)

# # Check if the molecule object was created successfully
# if mol is None:
#     print("Unable to create molecule object from the SMILES string.")

# # Get the clusters of atoms in the molecule
# clusters = Chem.GetMolFrags(mol, asMols=True)

# # Generate a SMILES string for each cluster of atoms
# for i, cluster in enumerate(clusters):
#     # Generate a SMILES string for the cluster
#     smiles = Chem.MolToSmiles(cluster)

#     # Print the SMILES string and cluster number
#     print("Cluster {} SMILES: {}".format(i+1, smiles))

575

In [415]:
import requests
from bs4 import BeautifulSoup

# Make a GET request to Sci-Hub with the DOI as the parameter
response = requests.get("https://sci-hub.ru/10.1016/S0166-1280(02)00027-1")

# Parse the HTML content of the response using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Extract the text of the article from the BeautifulSoup object
article_text = soup.get_text()

# Extract the value of the specific tag you want (replace 'tag' with the actual tag name)
tag_value = soup.find('title').get_text()

# Print the article text and the value of the specific tag
print(tag_value)


Sci-Hub | A theoretical and experimental study on the solubility, dissolution rate, structure and dipolar moment of flavone in ethanol. Journal of Molecular Structure: THEOCHEM, 585(1-3), 1–13 | 10.1016/S0166-1280(02)00027-1


In [416]:
# Просто пример вывода
requests.get("https://sci-hub.ru/10.1016/S0166-1280(02)00027-1").text

'<!DOCTYPE html>\n<html>\n    <head>\n        <title>Sci-Hub | A theoretical and experimental study on the solubility, dissolution rate, structure and dipolar moment of flavone in ethanol. Journal of Molecular Structure: THEOCHEM, 585(1-3), 1–13 | 10.1016/S0166-1280(02)00027-1</title>\n        <meta charset="UTF-8">\n        <meta name="viewport" content="width=device-width">\n        <script src="/scripts/jquery-3.6.0.min.js"></script>\n    </head>\n    <body>\n    <script type = "text/javascript">\n        \n        var menu;\n        var article;\n        var position;\n        function rollupi()\n        {\n            position = position - 2;\n            menu.style.left = position.toString() + \'vw\';\n            article.style.left = (position + 20).toString() + \'vw\';\n            article.style.width = (80 - position).toString() + \'vw\';\n            if (position > -20)\n                setTimeout(rollupi, 33);\n            else\n                document.getElementById(\'roll

In [417]:
# на всякий случай, если есть, избавляемся от пробелов по краям строк
df["DOI"] = df["DOI"].apply(lambda x: x.strip())

In [ ]:
# Обраюотка DOI
# Если есть пробелы, то начинаем обработку 

# индексы строк с пробелами в DOI
index_rows= df[df["DOI"].apply(lambda x: " " in x)].index

slice_wrong_DOIs = df.iloc[index_rows,column_index_dic["DOI"]]
correct_slice_DOIs = []
for doi in slice_wrong_DOIs.values:
    try:
        correct_doi = []
        for symbol in doi:
            correct_doi.append(symbol)
            # все DOI длинее 11 символов)
            if len(correct_doi)>11:
                response = requests.get(f"https://sci-hub.ru/{''.join(correct_doi)}")
                tag_value = BeautifulSoup(response.content, 'html.parser').find('title').get_text()
                if tag_value != "Sci-Hub: статья не найдена":
                    correct_slice_DOIs.append("".join(correct_doi))
                    break
    except:
        print("nothing")

# Вставляем исправленные значения в DOI
df.iloc[index_rows,column_index_dic["DOI"]] = correct_slice_DOIs

In [73]:
# Проверяем, какие статьи не ищутся и вытаскиваем индекс строк, таких статей
responces = []
doi_problems = []
for doi in df["DOI"]:
    response = requests.get(f"https://sci-hub.ru/{doi}")
    tag_value = BeautifulSoup(response.content, 'html.parser').find('title').get_text()
    if tag_value == "Sci-Hub: статья не найдена":
        doi_problems.append(doi)
    responces.append(tag_value)
    

In [418]:
# Проблемнеы DOIs, есть дубликаты - удаляем
doi_problems = set(doi_problems)

In [419]:
# Собственно проблемные DOIs

# План такой: убрать слова после цифр, эти DOIs прогнать через sci-hub
# потом остальные еще через что-то найти статьи 
# обновлять journal, title и date через crossref

processed_doi_problems = dict()
for doi in doi_problems:
    uppers = [l for l in doi[::-1] if l.isupper()]
    if len(uppers)>0: 
        processed_doi_problems[doi] = doi[:doi.find(uppers[0])]
    else:
        processed_doi_problems[doi] = doi

In [420]:
# Проверяем обработанные статьи ячейкой выше processed_doi_problems_part1
# Все прошли проверку но некоторые статьи из processed_doi_problems_part2 не прошли ее
import requests

problematic_doi = []
for doi in list(processed_doi_problems.values()):
    response = requests.get(f"https://api.crossref.org/works/{doi}")
    if response.ok:
        data = response.json()["message"]
        print(f"Title: {data['title'][0]}")
        print(f"Authors: {', '.join(author['given'] + ' ' + author['family'] for author in data['author'])}")
        print(f"Publication Date: {data['created']['date-parts'][0][0]}")
        print(f"Journal: {data['container-title'][0]}")
        print(f"DOI: {data['DOI']}")
    else:
        problematic_doi.append(doi)
        print("DOI not found.")

Title: Design and optimization of diamond-shaped biosensor using photonic crystal nano-ring resonator
Authors: Saeed Olyaee, Ahmad Mohebzadeh Bahabady
Publication Date: 2015
Journal: Optik
DOI: 10.1016/j.ijleo.2015.06.037
Title: Difluorinated analogues of shikimic acid
Authors: Lovely Begum, Julian M Box, Michael G.B Drew, Laurence M Harwood, Jane L Humphreys, David J Lowes, Gareth A Morris, Perrine M Redon, Francine M Walker, Roger C Whitehead
Publication Date: 2003
Journal: Tetrahedron
DOI: 10.1016/s0040-4020(03)00697-5
Title: Reflectivity of 88% for four-period hybrid Bragg mirror from spin coating process
Authors: L. Ajith DeSilva, Raghuveer Gadipalli, Anthony Donato, T.M.W.J. Bandara
Publication Date: 2017
Journal: Optik
DOI: 10.1016/j.ijleo.2017.11.048
Title: Gas permeation in thin films of “high free-volume” glassy perfluoropolymers: Part II. CO2 plasticization and sorption
Authors: Rajkiran R. Tiwari, Zachary P. Smith, Haiqing Lin, B.D. Freeman, D.R. Paul
Publication Date: 2014

In [421]:
# Проблемные статьи))
print(problematic_doi)

['10.3389/fpls.2014.00', '10.1038/srep28']


In [422]:
# Заменяем старые DOIs новыми, 
# но остается два неразрешенных DOIs - ['10.1038/srep28', '10.3389/fpls.2014.00']
for key,value in processed_doi_problems.items():
    df.loc[df["DOI"]==key,"DOI"] = value

In [423]:
# ['10.1038/srep28', '10.3389/fpls.2014.00'] удаляем эти DOIs (строки)
for doi_prob in problematic_doi:
    df = df.drop(df[df["DOI"]==doi_prob].index)

In [432]:
# Заполянем пропуски в (Date, Journal, Title)
# Индексы пропусков у них одинаковые

# Индексы пропусков у них одинаковые
missed_index = df[df["Journal"].isna()].index

In [434]:
# Создаем список, который потом заполнить пропуски
problematic_doi_2 = []
values_fill = []
for index in missed_index:
    response = requests.get(f"https://api.crossref.org/works/{df.iloc[index,column_index_dic['DOI']]}")
    if response.ok:
        data = response.json()["message"]
        values_fill.append([data['created']['date-parts'][0][0],
                            data['container-title'][0],data['title'][0]])
    else:
        problematic_doi_2.append(df.iloc[index,column_index_dic['DOI']])
        print("DOI not found.")

DOI not found.
DOI not found.
DOI not found.
DOI not found.
DOI not found.


In [436]:
# Удаляем эти DOIs (строки)
for doi_prob in problematic_doi_2:
    df = df.drop(df[df["DOI"]==doi_prob].index)

In [459]:
# Индексы пропусков у них одинаковые
missed_index = df[df["Journal"].isna()].index

In [460]:
# Создаем список, который потом заполнить пропуски (повторно, так как удалили некоторые строки)
problematic_doi_2 = []
values_fill = []
for index in missed_index:
    response = requests.get(f"https://api.crossref.org/works/{df.iloc[index,column_index_dic['DOI']]}")
    if response.ok:
        data = response.json()["message"]
        values_fill.append([data['created']['date-parts'][0][0],
                            data['container-title'][0],data['title'][0]])
    else:
        problematic_doi_2.append(df.iloc[index,column_index_dic['DOI']])
        print("DOI not found.")

In [469]:
# Заполянем пропуски в (Date, Journal, Title)
df.loc[missed_index,"Date"] = [x[0] for x in values_fill]
df.loc[missed_index,"Journal"] = [x[1] for x in values_fill]
df.loc[missed_index,"Title"] = [x[2] for x in values_fill]

## Заполняем пропуски в SMILES (через mol)

In [ ]:
# Удаляем дубликаты строк по DOI и Name и measurement_method и normalised_name и specifier
df = df.drop_duplicates(subset=["DOI","Name"])